<a href="https://colab.research.google.com/github/sungsushi/msci/blob/main/leaner_model/No_edges_paths.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# paths from not having to make edges!

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import copy 
import os
import time 
from google.colab import files
import copy 

def get_intercept(point): # generates 1e4 points and adj list in 3 sec
    return point[0] + point[1]

def gen_graph(N, seed=False): 
    # using box space condition
    if seed:
        np.random.seed(seed)
    #generate random coords in (0,1):
    x = np.random.random(N-2)
    y = np.random.random(N-2)

    
    
    unrotpoints = np.array([x,y])
    unrotpoints = unrotpoints[:,get_intercept(unrotpoints).argsort()].T # sort by ascending in causal order

    
    unrotpoints = np.insert(unrotpoints, [0], 0, axis=0) # add the (0,0) node to keep everything causal
    unrotpoints = np.insert(unrotpoints, [len(unrotpoints)], [1, 1], axis=0) # add the sink node
    
    points = np.array([unrotpoints[:,0]*np.cos(np.pi/4) - unrotpoints[:,1]*np.sin(np.pi/4), 
                     unrotpoints[:,1]*np.sin(np.pi/4) + unrotpoints[:,0]*np.cos(np.pi/4)]).T
    return points # only outputs coordinates sorted in time coordinate


In [ ]:

def get_intercept(point): # generates 1e4 points and adj list in 3 sec
    return point[0] + point[1]

def gen_graph_adj_2(N, seed=False): 
    # using box space condition
    if seed:
        np.random.seed(seed)
    #generate random coords in (0,1):
    x = np.random.random(N-2)
    y = np.random.random(N-2)

    
    
    unrotpoints = np.array([x,y])
    unrotpoints = unrotpoints[:,get_intercept(unrotpoints).argsort()].T # sort by ascending in causal order

    
    unrotpoints = np.insert(unrotpoints, [0], 0, axis=0) # add the (0,0) node to keep everything causal
    unrotpoints = np.insert(unrotpoints, [len(unrotpoints)], [1, 1], axis=0) # add the sink node
    
    adj = []
    for i in range(N): # need all adjacencies for algorithm but do we really?
        x_cond = (unrotpoints[i][0] - unrotpoints[i+1:][:,0]) < 0
        y_cond = (unrotpoints[i][1] - unrotpoints[i+1:][:,1]) < 0
        indices = np.where((x_cond * y_cond)==True)[0]
        adj.append(list(indices+i+1))

    points = np.array([unrotpoints[:,0]*np.cos(np.pi/4) - unrotpoints[:,1]*np.sin(np.pi/4), 
                     unrotpoints[:,1]*np.sin(np.pi/4) + unrotpoints[:,0]*np.cos(np.pi/4)]).T
    return points, adj

def random_path(adj_list, seed=None):
    if seed:
        np.random.seed(seed)
    N = len(adj_list)
    i = 0
    path = [0]
    while i < N-1:
        #print(i)
        j = np.random.choice(adj_list[i])
        path.append(j)
        i = j
    return path



def greedy_path(adj):
    # adj is the adjacency list of the network. 
    # points is the coordinates of those nodes
    path = [0] #start with node zero
    current_node = 0
    while current_node != len(adj)-1:
        next_node = adj[current_node][0] # because we have already time sorted it, 
                                    #0th element should be the closest in time. 
        path.append(next_node)
        current_node = next_node
    return path


def longest_path(adj_list): # origin node is taken to be index (and label) 0.
    N = len(adj_list) # N = no. of nodes in the graph
    
    paths = [[] for i in range(N)]
    paths[0].append(0)  # the path lengths of the shortest from the origin node.
    
    for i in range(N):
        _list = adj_list[i] 
        for j in _list: # consider daughter vertices from node i. 
            plength = len(paths[i]) +1
            if plength > len(paths[j]): # if new path is longer than existing path
                paths[j] = paths[i] + [j] # replace old path with the new path
    return paths # prints an array of longest paths  whose index corresponds to node label.



In [ ]:

def ptgpath_points(points):
    # assume points are sorted by its time coordinate
    current_node = 0
    path = [0]
    N = len(points)
    while current_node < N - 1:
        #print(current_node)
        c_node_coord = points[current_node]
        domain = points[current_node + 1:] # all nodes after current_node in time coordinate
        propertimes = (c_node_coord[0] - domain[:,0])**2 - (c_node_coord[1] - domain[:,1])**2
        invpt = 1/propertimes # inverse pt: numbers near 0 become large in magnitude
        index = np.argmin(invpt) # index of the smallest negative inverse pt (i.e. least negative pt)
        current_node += 1+ index # new current node
        path.append(current_node) # add the new current node to the path
    pathpoints = points[path]
    return pathpoints # array of coordinates in the path

def random_path_points(points, seed=None):
    # points is the coordinates of nodes sorted in time coord
    if seed:
        np.random.seed(seed)
    N = len(points)
    current_node = 0
    path = [0]
    while current_node < N-1:
        #print(current_node)
        c_node_coord = points[current_node]
        domain = points[current_node + 1:] # all nodes after current_node in time coordinate
        propertimes = (c_node_coord[0] - domain[:,0])**2 - (c_node_coord[1] - domain[:,1])**2
        causal = np.where(propertimes < 0)[0] # which indicies of propertimes are causal?
        index = np.random.choice(causal) # choose one of the indices at random
        current_node += 1 + index # new current node
        path.append(current_node) # add the new current node to the path
    
    pathpoints = points[path]
    return path

def greedy_path_points(points):
    # points is the coordinates of nodes sorted in time coord
    path = [0] #start with node zero
    current_node = 0
    N = len(points)
    while current_node < N-1:
        c_node_coord = points[current_node]
        domain = points[current_node + 1:] # all nodes after current_node in time coordinate
        propertimes = (c_node_coord[0] - domain[:,0])**2 - (c_node_coord[1] - domain[:,1])**2
        causal = np.where(propertimes < 0)[0] # which indicies of propertimes are causal?
        index = min(causal) # choose one of the indices at random
        current_node += 1 + index # new current node
        path.append(current_node) # add the new current node to the path
    pathpoints = points[path]
    return path



def longest_path_points(points): # origin node is taken to be index (and label) 0.
    N = len(points) # N = no. of nodes in the graph
    
    paths = [[] for i in range(N)]
    paths[0].append(0)  # the path lengths of the shortest from the origin node.
    
    for i in range(N):
        c_node_coord = points[i]
        domain = points[i + 1:] # all nodes after current_node in time coordinate
        propertimes = (c_node_coord[0] - domain[:,0])**2 - (c_node_coord[1] - domain[:,1])**2
        causal = np.where(propertimes < 0)[0] # which indicies of propertimes are causal?
        _list = i + 1 + causal 
        for j in _list: # consider daughter vertices from node i. 
            plength = len(paths[i]) +1
            if plength > len(paths[j]): # if new path is longer than existing path
                paths[j] = paths[i] + [j] # replace old path with the new path
    return paths # prints an array of longest paths  whose index corresponds to node label.



## random 

In [ ]:
N = int(1e4) 
points = gen_graph(N, seed=1)

#print(ptgpath_points(points))
print(random_path_points(points, seed = 1))

points, adj = gen_graph_adj_2(N, seed=1)
print(random_path(adj, seed=1))

[0, 236, 6050, 9456, 9584, 9989, 9997, 9999]
[0, 236, 6050, 9456, 9584, 9989, 9997, 9999]


## time-greedy

In [ ]:
N = int(2e4) 
points = gen_graph(N, seed=1)

#print(ptgpath_points(points))
print(greedy_path_points(points))

[0, 1, 3, 4, 7, 9, 23, 36, 53, 54, 62, 82, 106, 126, 141, 165, 180, 218, 228, 249, 253, 261, 285, 320, 351, 393, 424, 455, 494, 506, 544, 563, 592, 613, 690, 719, 825, 863, 876, 937, 955, 967, 997, 1052, 1075, 1091, 1140, 1195, 1236, 1358, 1477, 1505, 1560, 1675, 1810, 1905, 1912, 1958, 2000, 2056, 2063, 2174, 2303, 2377, 2400, 2480, 2607, 2696, 2735, 2810, 2889, 2967, 3094, 3203, 3294, 3362, 3511, 3712, 3724, 3809, 4049, 4091, 4170, 4257, 4379, 4497, 4564, 4729, 4872, 5031, 5158, 5360, 5470, 5503, 5649, 5787, 5990, 6096, 6260, 6352, 6430, 6484, 6604, 6712, 6770, 6886, 6984, 7170, 7353, 7405, 7512, 7558, 7726, 7787, 7904, 8222, 8464, 8848, 8947, 9016, 9289, 9447, 9645, 9845, 10120, 10223, 10389, 10498, 10719, 10902, 11090, 11204, 11280, 11335, 11394, 11410, 11623, 11773, 11813, 12100, 12220, 12414, 12488, 12592, 12856, 12965, 13220, 13304, 13492, 13645, 13785, 13831, 13911, 13975, 14180, 14468, 14637, 14670, 14743, 14838, 14946, 15049, 15177, 15325, 15484, 15540, 15698, 15792, 15877, 1

In [ ]:
N = int(2e4) 

points, adj = gen_graph_adj_2(N, seed=1)
print(greedy_path(adj))

[0, 1, 3, 4, 7, 9, 23, 36, 53, 54, 62, 82, 106, 126, 141, 165, 180, 218, 228, 249, 253, 261, 285, 320, 351, 393, 424, 455, 494, 506, 544, 563, 592, 613, 690, 719, 825, 863, 876, 937, 955, 967, 997, 1052, 1075, 1091, 1140, 1195, 1236, 1358, 1477, 1505, 1560, 1675, 1810, 1905, 1912, 1958, 2000, 2056, 2063, 2174, 2303, 2377, 2400, 2480, 2607, 2696, 2735, 2810, 2889, 2967, 3094, 3203, 3294, 3362, 3511, 3712, 3724, 3809, 4049, 4091, 4170, 4257, 4379, 4497, 4564, 4729, 4872, 5031, 5158, 5360, 5470, 5503, 5649, 5787, 5990, 6096, 6260, 6352, 6430, 6484, 6604, 6712, 6770, 6886, 6984, 7170, 7353, 7405, 7512, 7558, 7726, 7787, 7904, 8222, 8464, 8848, 8947, 9016, 9289, 9447, 9645, 9845, 10120, 10223, 10389, 10498, 10719, 10902, 11090, 11204, 11280, 11335, 11394, 11410, 11623, 11773, 11813, 12100, 12220, 12414, 12488, 12592, 12856, 12965, 13220, 13304, 13492, 13645, 13785, 13831, 13911, 13975, 14180, 14468, 14637, 14670, 14743, 14838, 14946, 15049, 15177, 15325, 15484, 15540, 15698, 15792, 15877, 1

## longest

In [ ]:
N = int(2e4) 
points = gen_graph(N, seed=1)

#print(ptgpath_points(points))
print(len(longest_path_points(points)[-1]))


333


In [ ]:
N = int(2e4) 

points, adj = gen_graph_adj_2(N, seed=1)
print(len(longest_path(adj)[-1]))